# Capstone Project: Intelligent Customer Support Router
### Track: Concierge Agents (Customer Service)

**Project Overview**
This project implements a **Hierarchical Multi-Agent System** using the **Google Agent Development Kit (ADK)**. Unlike traditional chatbots that rely on rigid keyword matching (e.g., `if "refund" in text`), this system uses a semantic **Router-Specialist architecture**.

**Key Technical Features:**
1.  **Semantic Routing:** A Root Agent powered by `gemini-2.5-flash-lite` intelligently determines user intent and delegates tasks to the correct specialist.
2.  **Functional Tools:** Sub-agents are equipped with Python-based tools (`Dict` return types) to perform logic and simulate database lookups.
3.  **Observability:** Integrated `LoggingPlugin` provides real-time tracing of the agent's thought process, tool calls, and state transitions.

**Architecture:**
* **Root Agent:** Triage & Routing
* **Billing Specialist:** Handles refunds & invoices (Equipped with `check_refund_status`)
* **Tech Support:** Handles device troubleshooting (Equipped with `troubleshoot_device`)

## Step 1: Secure Authentication
To interact with the Gemini API, we securely retrieve the API key from Kaggle Secrets. This ensures our credentials are not exposed in the codebase.

In [1]:
# 1. AUTHENTICATION
import os
from kaggle_secrets import UserSecretsClient

try:
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = api_key
    print("API Key loaded successfully!")
except Exception as e:
    print("Error loading API Key. Please check your Secrets.")
    raise e

API Key loaded successfully!


## Step 2: Setup & Imports
We import the necessary components from the **Google ADK**.
* **`LlmAgent`**: The core class for creating AI agents.
* **`AgentTool`**: Allows agents to call *other* agents as tools (crucial for hierarchical routing).
* **`LoggingPlugin`**: Enables the "Observability" requirement by tracing agent actions.

In [2]:
# 2. SETUP & IMPORTS
from typing import Dict, Any
from google.adk.agents import LlmAgent
from google.adk.tools import AgentTool
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import LoggingPlugin
from google.genai import types

## Step 3: Defining Functional Tools
Agents need ways to interact with the world. Here, we define two Python functions that act as tools.
* **`check_refund_status`**: Simulates a database lookup for order statuses.
* **`troubleshoot_device`**: Provides logic branches for technical issues.

*Note: We use strict type hinting (`Dict[str, Any]`) and detailed docstrings. The Agents read these docstrings to understand **when** and **how** to use the tools.*

In [3]:
# 3. DEFINE TOOLS 
def check_refund_status(order_id: str) -> Dict[str, Any]:
    """
    Checks the refund status for a specific order ID.
    Use this when a user asks about the status of their refund.
    """
    mock_db = {
        "ORD-123": "Processed",
        "ORD-456": "Pending Approval",
        "ORD-789": "Declined"
    }
    status = mock_db.get(order_id, "Not Found")
    return {"order_id": order_id, "status": status}

def troubleshoot_device(device_type: str, issue: str) -> Dict[str, Any]:
    """
    Provides technical troubleshooting steps based on the device and issue.
    Use this for technical support queries.
    """
    steps = f"To fix {issue} on {device_type}: 1. Restart device. 2. Check updates. 3. Reset settings."
    return {"steps": steps}

## Step 4: Creating Specialist Sub-Agents
We create two distinct agents using the **`gemini-2.5-flash-lite`** model. Each agent has a specific "persona" and a restricted set of tools. This "Separation of Concerns" reduces hallucinations and improves accuracy.

1.  **`billing_agent`**: Only handles financial queries.
2.  **`tech_agent`**: Only handles hardware/software queries.

In [4]:
# 4. DEFINE SPECIALIST AGENTS

# Billing Specialist
billing_agent = LlmAgent(
    name="BillingSpecialist",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
    You are a Billing Support Specialist.
    1. Use the 'check_refund_status' tool if the user provides an Order ID.
    2. IMPORTANT: After using the tool, you MUST summarize the status in a final text response.
    """,
    tools=[check_refund_status]
)


In [5]:
# Technical Support Specialist
tech_agent = LlmAgent(
    name="TechSupport",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
    You are a Technical Support Engineer.
    1. Use the 'troubleshoot_device' tool to provide specific steps.
    2. CRITICAL: You MUST read the output from the tool and speak it back as your final answer.
    3. Do not just call the tool and stop.
    """,
    tools=[troubleshoot_device]
)

## Step 5: The Root Router (Triage Agent)
This is the "brain" of the operation. The **Root Agent** does not have direct access to the refund database or troubleshooting logic. Instead, it has access to the **Specialist Agents** via `AgentTool`.

Its instruction is purely **Orchestration**: it analyzes the user's request and routes it to the specialist best suited to handle it.

In [6]:
# 5. DEFINE ROOT ROUTER AGENT

root_agent = LlmAgent(
    name="CustomerSupportRouter",
    model=Gemini(model="gemini-2.5-flash-lite"), 
    instruction="""
    You are the Head of Customer Support (Triage).
    Your job is to route the user to the correct specialist.
    
    - If the issue is about money, refunds, or orders -> Call 'BillingSpecialist'.
    - If the issue is technical, broken devices, or bugs -> Call 'TechSupport'.
    - If the issue is general (like "hello"), answer it yourself politely.
    
    When the specialist returns with an answer, repeat that answer to the user.
    """,
    tools=[AgentTool(billing_agent), AgentTool(tech_agent)] 
)


## Step 6: Execution & Observability
Finally, we run the system using the `InMemoryRunner`. We attach the **`LoggingPlugin`** to the runner, which will output the agent's "Thought Process" (logs) to the console.

We test the system with three distinct queries to demonstrate:
1.  **General Chat:** Handling "Hello" locally.
2.  **Billing Logic:** Routing to the Billing Specialist for a refund.
3.  **Tech Support:** Routing to the Tech Specialist for a broken device.

In [7]:
# 6. RUN WITH OBSERVABILITY

async def main():
    print("AI Support System Initialized...\n")
    
    runner = InMemoryRunner(
        agent=root_agent,
        plugins=[LoggingPlugin()]
    )

    test_queries = [
        "Hello, who are you?", 
        "I need a refund for order ORD-456",
        "My iPhone camera is not working"
    ]

    for query in test_queries:
        print(f"\n ----> User: {query}")
        print("-" * 40)
        await runner.run_debug(query) 
        print("-" * 40)


In [8]:
# Run the async main function
await main()

AI Support System Initialized...


 ----> User: Hello, who are you?
----------------------------------------

 ### Created new session: debug_session_id

User > Hello, who are you?
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-35254b0b-45e9-4694-bfad-86fe5eaed5c2
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: CustomerSupportRouter
[logging_plugin]    User Content: text: 'Hello, who are you?'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-35254b0b-45e9-4694-bfad-86fe5eaed5c2
[logging_plugin]    Starting Agent: CustomerSupportRouter
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: CustomerSupportRouter
[logging_plugin]    Invocation ID: e-35254b0b-45e9-4694-bfad-86fe5eaed5c2
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: CustomerSup

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: CustomerSupportRouter
[logging_plugin]    Content: function_call: BillingSpecialist
[logging_plugin]    Token Usage - Input: 240, Output: 22
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 60100bf6-0753-4815-b9c6-9e885e98fc3a
[logging_plugin]    Author: CustomerSupportRouter
[logging_plugin]    Content: function_call: BillingSpecialist
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['BillingSpecialist']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: BillingSpecialist
[logging_plugin]    Agent: CustomerSupportRouter
[logging_plugin]    Function Call ID: adk-1a8203cb-1748-4e52-ad7f-15a591cd6abf
[logging_plugin]    Arguments: {'request': 'Refund for order ORD-456'}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-69600295-e6ed-4b1e-8072-77c9a2fd1410
[logging_plugin]    Session ID: 5eb995e7-5fde-43a0-9ab3-96d8e74c211a
[logging_plugin]    Use

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: BillingSpecialist
[logging_plugin]    Content: function_call: check_refund_status
[logging_plugin]    Token Usage - Input: 147, Output: 23
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: ac21efa9-affa-47eb-8cfe-061328cc12b0
[logging_plugin]    Author: BillingSpecialist
[logging_plugin]    Content: function_call: check_refund_status
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['check_refund_status']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: check_refund_status
[logging_plugin]    Agent: BillingSpecialist
[logging_plugin]    Function Call ID: adk-f0d4ad93-3115-4325-9010-ee13ded31264
[logging_plugin]    Arguments: {'order_id': 'ORD-456'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: check_refund_status
[logging_plugin]    Agent: BillingSpecialist
[logging_plugin]    Function Call ID: adk-f0d4ad93-3115-4325-9010-ee13ded31264
[logging_plugin]  

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: CustomerSupportRouter
[logging_plugin]    Content: function_call: TechSupport
[logging_plugin]    Token Usage - Input: 328, Output: 18
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: c98cf20d-cab1-41e6-a5d3-84b484f48a56
[logging_plugin]    Author: CustomerSupportRouter
[logging_plugin]    Content: function_call: TechSupport
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['TechSupport']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: TechSupport
[logging_plugin]    Agent: CustomerSupportRouter
[logging_plugin]    Function Call ID: adk-62321b08-489f-4a68-857f-649ab4e0e912
[logging_plugin]    Arguments: {'request': 'iPhone camera is not working'}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-e397d642-3bb6-4c6a-b2fc-2dd02036661f
[logging_plugin]    Session ID: 93549be3-f4d6-42d5-9c13-19023be4d206
[logging_plugin]    User ID: debug_user_id


[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: TechSupport
[logging_plugin]    Content: function_call: troubleshoot_device
[logging_plugin]    Token Usage - Input: 159, Output: 27
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 3e7df542-e7d5-4e5a-8a7f-9a65846d2342
[logging_plugin]    Author: TechSupport
[logging_plugin]    Content: function_call: troubleshoot_device
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['troubleshoot_device']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: troubleshoot_device
[logging_plugin]    Agent: TechSupport
[logging_plugin]    Function Call ID: adk-a34cc7df-5ba3-40c4-8e23-e1d0c5a6b3e8
[logging_plugin]    Arguments: {'issue': 'Camera is not working', 'device_type': 'iPhone'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: troubleshoot_device
[logging_plugin]    Agent: TechSupport
[logging_plugin]    Function Call ID: adk-a34cc7df-5ba3-40c4-8e23-e1d0c5a6b3e8
[loggi

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: TechSupport
[logging_plugin]    Content: function_call: troubleshoot_device
[logging_plugin]    Token Usage - Input: 229, Output: 27
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 112f7b07-92f8-415c-aabf-0385d88c4825
[logging_plugin]    Author: TechSupport
[logging_plugin]    Content: function_call: troubleshoot_device
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['troubleshoot_device']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: troubleshoot_device
[logging_plugin]    Agent: TechSupport
[logging_plugin]    Function Call ID: adk-67d80d89-f51a-45a4-8c95-8f6a5dc9bb2c
[logging_plugin]    Arguments: {'device_type': 'iPhone', 'issue': 'Camera is not working'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: troubleshoot_device
[logging_plugin]    Agent: TechSupport
[logging_plugin]    Function Call ID: adk-67d80d89-f51a-45a4-8c95-8f6a5dc9bb2c
[loggi